In [ ]:
#  сборщик мусора
import gc
# системные команды
import sys
#ошибки
import warnings
# joblib- это набор инструментов для облегчения конвейерной обработки в Python . В частности:
# прозрачное кеширование функций на диске и ленивая переоценка (шаблон Memoize
# простые простые параллельные вычисления
from joblib import Parallel, delayed
# Пути к объектно-ориентированной файловой системе
import os
from pathlib import Path
#  виджеты, представляют собой интерактивные HTML-виджеты для записных книжек Jupyter и ядра IPython.
import ipywidgets as widgets

# стандартные
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# TensorFlow — открытая программная библиотека для машинного обучения, разработанная компанией Google для решения задач построения и тренировки нейронной сети 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.experimental.preprocessing import StringLookup
from datetime import timedelta

from functools import reduce
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import lightgbm as lgbm

In [ ]:
from sklearn.preprocessing import LabelEncoder
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

### INPUT DATA

In [ ]:
player_target_stats = pd.read_csv('../input/player-target-stats/player_target_stats.csv')

In [ ]:
TRAIN_DIR = Path('../input/mlb-pdef-train-dataset')
rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')
scores = pd.read_pickle(TRAIN_DIR / 'playerBoxScores_train.pkl')
awards_train = pd.read_pickle(TRAIN_DIR / 'awards_train.pkl')
playerTwitter = pd.read_pickle(TRAIN_DIR / 'playerTwitterFollowers_train.pkl')
teamTwitter = pd.read_pickle(TRAIN_DIR / 'teamTwitterFollowers_train.pkl')
#transactions = pd.read_pickle(TRAIN_DIR / 'transactions_train.pkl')
#standings = pd.read_pickle(TRAIN_DIR / 'standings_train.pkl')

In [ ]:
data_dir = Path('../input/mlb-player-digital-engagement-forecasting/')

df_names = ['seasons', 'teams', 'players', 'awards']
#  читаем файлы создаем ДФ по имени файла
for name in df_names:
    globals()[name] = pd.read_csv(data_dir / f"{name}.csv")

### Pre procesing

In [ ]:
seasons['allStarDate'].fillna(value=pd.to_datetime('1/1/2015'), inplace=True) 
#seasons.fillna('2018-03-29')
seasons_col = seasons.columns[1:]
for col in seasons_col :
    seasons[col] = pd.to_datetime(seasons[col]).values.astype('datetime64[D]')
    #seasons[col] = pd.to_datetime(seasons[col], format = '%Y%m%d')

In [ ]:
#seasons

In [ ]:
# кодировка категориальных в числовые
players['label_playerId'] = players['playerId']
# df_coding_col = ['label_playerId']
# train = MultiColumnLabelEncoder(columns = df_coding_col).fit_transform(train)
df_coding_columns = ['primaryPositionName','birthCountry' , 'label_playerId']
players = MultiColumnLabelEncoder(columns = df_coding_columns).fit_transform(players)

In [ ]:
time_col = ['mlbDebutDate','DOB']
for col in time_col :
    players[col] = pd.to_datetime(players[col]).values.astype('datetime64[D]')

In [ ]:
from datetime import datetime
from datetime import date

In [ ]:
players['birthday_year'] = pd.DatetimeIndex(players['DOB']).year
players['Debut_year'] = pd.DatetimeIndex(players['mlbDebutDate']).year

In [ ]:
#   пока только количество лет
def udate_year_old(df):
    df['date_year'] = pd.to_datetime(df['date'], format = '%Y%m%d')
    df['date_year'] = pd.DatetimeIndex(df['date_year']).year
    newColumn = lambda x: x['date_year'] - x['birthday_year']
    df['year_old'] = df.apply(newColumn, axis=1)
    
    
    newColumn2 = lambda x: x['date_year'] - x['Debut_year']
    df['year_play'] = df.apply(newColumn2, axis=1) 
    return df

In [ ]:
players.fillna(0, inplace=True)

In [ ]:
#players.info()

In [ ]:
# Отбираем только игроков участвующих в замерах playerForTestSetAndFuturePreds = True
pids_test = players.playerId.loc[ players.playerForTestSetAndFuturePreds.fillna(False)].astype(str)

In [ ]:
scores.fillna(0, inplace=True)
scores = scores.groupby(['playerId', 'date']).sum().reset_index()

In [ ]:
#   пока только Сизон Да/нет  и этап сизона
def udate_season(df):
    df['year'] = df.index.year
    df['month'] = df.index.month
    #df['date'] = df.index.to_timestamp()
    df['date_id'] = df.index.to_timestamp()
    #display(df)
    #df['date_id'] = df['date']
    
    # Собираем вместе сезоны и даты
    dates_with_info = pd.merge( df, seasons,  left_on = 'year', right_on = 'seasonId')
    # Create date index
#     dates_with_info['period'] = dates_with_info['date']
#     dates_with_info['period'] = pd.to_datetime(train['period'], format = '%Y%m%d')
#     dates_with_info['date'] = pd.PeriodIndex(dates_with_info.date, freq='D')
    
    dates_with_info['inSeason'] = ( dates_with_info['date_id'].between( dates_with_info['regularSeasonStartDate'], dates_with_info['postSeasonEndDate'],
    inclusive = True ) )
    # Separate dates into different parts of MLB season
    dates_with_info['seasonPart'] = np.select(
      [
        dates_with_info['date_id'] < dates_with_info['preSeasonStartDate'], 
        dates_with_info['date_id'] < dates_with_info['regularSeasonStartDate'],
        dates_with_info['date_id'] <= dates_with_info['lastDate1stHalf'],
        dates_with_info['date_id'] < dates_with_info['firstDate2ndHalf'],
        dates_with_info['date_id'] <= dates_with_info['regularSeasonEndDate'],
        dates_with_info['date_id'] < dates_with_info['postSeasonStartDate'],
        dates_with_info['date_id'] <= dates_with_info['postSeasonEndDate'],
        dates_with_info['date_id'] > dates_with_info['postSeasonEndDate']
      ], 
      [  -1,1,2,3,4,2,1,-1 ], 
      default = np.nan
      )
    

#     [ 'Offseason', 'Preseason', 'Reg Season 1st Half','All-Star Break','Reg Season 2nd Half','Between Reg and Postseason','Postseason','Offseason' ], 
    
     
    
    #display(dates_with_info)
    dates_with_info = dates_with_info.drop(['preSeasonEndDate', 'regularSeasonStartDate','regularSeasonEndDate','lastDate1stHalf',
                  'allStarDate',       'firstDate2ndHalf', 'postSeasonStartDate','postSeasonEndDate', 'seasonStartDate','seasonEndDate',
                  'preSeasonStartDate','seasonId' ,'date_id', 'year', 'month'], axis =1 )
             
    
    df = dates_with_info
    #df = df.set_index('date')
    df = df.astype({'inSeason':'int32'})  
    df_col = ['inSeason','seasonPart']
    df = df.astype({name: np.float32 for name in df_col})
    del dates_with_info
    gc.collect();
    #display(df)
    return df

##### player awards

In [ ]:
# def udate_award(df):
#     # Выделяем организацияю награды и отбираем по ней
#     df['awar_org'] = df['awardName'].apply(lambda x: x.split(' ')[0])
#     df = df[(df.awar_org == 'AL') | (df.awar_org == 'NL')]
#     return df

In [ ]:
# awards_train = udate_award(awards_train)
#awards_train

##### playerTwitter

In [ ]:
# Добавляем период для стыковки с данными
playerTwitter['period'] = pd.to_datetime(playerTwitter['date'], format = '%Y%m%d')
playerTwitter['twit_year'] = playerTwitter['period'].dt.year
playerTwitter['twit_month'] = playerTwitter['period'].dt.month

In [ ]:
# расчет медианного значения
med_followers = pd.DataFrame(playerTwitter.groupby('playerId')['numberOfFollowers'].median().sort_values(ascending=False)).reset_index()
med_followers.columns = ['playerId', 'twit_med_f']

min_followers = pd.DataFrame(playerTwitter.groupby('playerId').agg({'numberOfFollowers':'min'})['numberOfFollowers']).reset_index()
min_followers.columns = ['playerId', 'twit_min_f']

max_followers = pd.DataFrame(playerTwitter.groupby('playerId').agg({'numberOfFollowers':'max'})['numberOfFollowers']).reset_index()
max_followers.columns = ['playerId', 'twit_max_f']

def udate_playTwiter(df):
    df = df.merge(med_followers , on =['playerId'], how = 'left')
    df = df.merge(min_followers , on =['playerId'], how = 'left')
    df = df.merge(max_followers , on =['playerId'], how = 'left')
    
    newColumn2 = lambda x: x['numberOfFollowers'] - x['twit_med_f']
    df['twit_trend'] = df.apply(newColumn2, axis=1) 
    return df

playerTwitter = udate_playTwiter(playerTwitter)
playerTwitter.head()


_index = (playerTwitter['date'] == 20210401)
playerTwitter_test = playerTwitter.loc[_index].reset_index(drop=True)

del med_followers ,min_followers ,max_followers
gc.collect();

# playerTwitter = playerTwitter.merge(med_followers , on =['playerId'], how = 'left')
# newColumn2 = lambda x: x['numberOfFollowers'] - x['med_f']
# playerTwitter['twit_trend'] = playerTwitter.apply(newColumn2, axis=1) 

## teamTwit

In [ ]:
# Добавляем период для стыковки с данными
teamTwitter.columns = ['date','teamId','teamName','accountName','twitterHandle','team_numberOfFollowers' ,'index']

teamTwitter['period'] = pd.to_datetime(teamTwitter['date'], format = '%Y%m%d')
teamTwitter['twit_year'] = teamTwitter['period'].dt.year
teamTwitter['twit_month'] = teamTwitter['period'].dt.month

In [ ]:
# расчет медианного значения
med_followers_team = pd.DataFrame(teamTwitter.groupby('teamId')['team_numberOfFollowers'].median().sort_values(ascending=False)).reset_index()
med_followers_team.columns = ['teamId', 'team_twit_med_f']

min_followers_team = pd.DataFrame(teamTwitter.groupby('teamId').agg({'team_numberOfFollowers':'min'})['team_numberOfFollowers']).reset_index()
min_followers_team.columns = ['teamId', 'team_twit_min_f']

max_followers_team = pd.DataFrame(teamTwitter.groupby('teamId').agg({'team_numberOfFollowers':'max'})['team_numberOfFollowers']).reset_index()
max_followers_team.columns = ['teamId', 'team_twit_max_f']

def udate_teamTwiter(df):
    df = df.merge(med_followers_team , on =['teamId'], how = 'left')
    df = df.merge(min_followers_team , on =['teamId'], how = 'left')
    df = df.merge(max_followers_team , on =['teamId'], how = 'left')
    
    newColumn2 = lambda x: x['team_numberOfFollowers'] - x['team_twit_med_f']
    df['team_twit_trend'] = df.apply(newColumn2, axis=1) 
    return df

teamTwitter = udate_teamTwiter(teamTwitter)
teamTwitter.head()


_index = (teamTwitter['date'] == 20210401)
teamTwitter_test = teamTwitter.loc[_index].reset_index(drop=True)

del med_followers_team ,min_followers_team ,max_followers_team
gc.collect();

In [ ]:
#playerTwitter_test

##### Transactions

In [ ]:
# trans_col = ['playerId','date','fromTeamId','toTeamId','typeCode']
# trans_train = transactions[trans_col]
# trans_train.columns = ['playerId','date','fromTeamId','toTeamId','tran_type']
# print(trans_train.shape)

# trans_train = trans_train[(trans_train.tran_type == 'TR')]
# print(trans_train.shape)

### standings

In [ ]:
stand_col = ['teamId','date','wins' ,'homeWins','awayWins','lastTenWins','oneRunWins','dayWins','nightWins','grassWins','divWins','xWinLossPct']

In [ ]:
# standings_train = standings[stand_col]

In [ ]:
# ros_stan =  rosters.merge(standings_train , on = ['teamId','date'] , how ='left')
# ros_stan.fillna(0, inplace=True )
# ros_stan.head()

## Target team

In [ ]:
rosters_tag = targets.merge(rosters, on=['playerId','date'], how='left')
team_tag_sum =  rosters_tag.groupby(['teamId','date'])['target1'].agg('sum').reset_index()
team_tag_sum.columns = ['teamId', 'date','targ_team']
team_tag_sum.head()

In [ ]:
team_mean =  team_tag_sum.groupby(['teamId'])['targ_team'].agg('mean').reset_index()
team_mean.columns = ['teamId','targ_team_mean']
team_mean.head()

In [ ]:
team_tag_sum = team_tag_sum.merge(team_mean , on= ['teamId'], how = 'left')
team_tag_sum.head()

### LGBM  DATA

In [ ]:
# this++++++++++++++++++ 
players_cols = ['playerId', 'primaryPositionName', 'label_playerId', 'birthday_year','Debut_year']
#++++++++++++++++++++++
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4' ,'date']
rosters_cols = ['playerId', 'teamId','status', 'date']
# scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
#        'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
#        'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
#        'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
#        'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
#        'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
#        'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
#        'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
#        'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
#        'groundOutsPitching', 'runsPitching', 'doublesPitching',
#        'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
#        'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
#        'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
#        'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
#        'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
#        'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
#        'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
#        'inheritedRunnersScored', 'catchersInterferencePitching',
#        'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
#        'assists', 'putOuts', 'errors', 'chances', 'date']
scores_cols = ['playerId', "hits",
    "strikeOuts",
    "homeRuns",
    "runsScored",
    "stolenBases",
    "strikeOutsPitching",
    "inningsPitched",
    "strikes",
    "flyOuts",
    "groundOuts",
    "errors",  'date']


#awards_cols = ['awar_org','playerId','date']
trans_col = ['playerId','date','fromTeamId','toTeamId','tran_type']

#playerTwitter_cols = ['playerId','numberOfFollowers', 'twit_year','twit_month','twit_trend']
playerTwitter_cols = ['playerId','numberOfFollowers', 'twit_year','twit_month','twit_med_f' , 'twit_min_f', 'twit_max_f','twit_trend']

stand_col = ['playerId','date','wins' ,'homeWins','awayWins','lastTenWins','oneRunWins']
# here++++++++++++++++++++++++++++++++
teamTwitter_cols = ['teamId','team_numberOfFollowers', 'twit_year','twit_month','team_twit_med_f' , 'team_twit_min_f', 'team_twit_max_f','team_twit_trend']
# here++++++++++++++++++++++++++++++++

In [ ]:
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')

In [ ]:
train['period'] = train['date']
train['period'] = pd.to_datetime(train['period'], format = '%Y%m%d')
# here++++++++++++++++++++++++
train['twit_year'] = train['period'].dt.year
train['twit_month'] = train['period'].dt.month
#+++++++++++++++++++++++++++++
train = train.set_index('period').to_period('D')
train = udate_season(train)

In [ ]:
# here++++++++++++++++++++++++++++++++
# Обогащаем наградами
# train = train.merge(awards_train[awards_cols], on=['playerId', 'date'], how='left')
# train['awar_org'].value_counts()
# here++++++++++++++++++++++++++++++++

In [ ]:
# here++++++++++++++++++++++++++++++++
# Обогащаем транзакциями
# train = train.merge(trans_train[trans_col], on=['playerId', 'date'], how='left')
# train['tran_type'].value_counts()
# here++++++++++++++++++++++++++++++++

In [ ]:
# Обогащаем турнирной таблицей
# train = train.merge(ros_stan[stand_col], on=['playerId', 'date'], how='left')
# train['homeWins'].value_counts()
# #'wins' ,'homeWins','awayWins','lastTenWins','oneRunWins','dayWins','nightWins','grassWins','divWins','xWinLossPct']
# train.fillna({'wins':0, 'homeWins':0 , 'awayWins':0 ,'lastTenWins':0 , 'dayWins':0 ,'nightWins':0,'grassWins':0,'divWins':0,'xWinLossPct':0 }, inplace=True)
# here++++++++++++++++++++++++++++++++

In [ ]:
#train =  udate_award(train)
#train.head()

In [ ]:
# Обогащаем twiterom play
train = train.merge(playerTwitter[playerTwitter_cols], on=['playerId', 'twit_year','twit_month'], how='left')
#train.fillna({'numberOfFollowers':0, 'twit_trend':0}, inplace=True)
train.fillna({'numberOfFollowers':0, 'twit_med_f':0 , 'twit_min_f':0 ,'twit_max_f':0 ,'twit_trend':0 }, inplace=True)
#train.fillna(0,inplace=True)
#train.head()

In [ ]:
# Обогащаем twiterom team
train = train.merge(teamTwitter[teamTwitter_cols], on=['teamId', 'twit_year','twit_month'], how='left')
#train.fillna({'numberOfFollowers':0, 'twit_trend':0}, inplace=True)
train.fillna({'team_numberOfFollowers':0, 'team_twit_med_f':0 , 'team_twit_min_f':0 ,'team_twit_max_f':0 ,'team_twit_trend':0 }, inplace=True)
#'team_twit_med_f' , 'team_twit_min_f', 'team_twit_max_f','team_twit_trend']
#train.fillna(0,inplace=True)
#train.head()
train['team_numberOfFollowers'].value_counts()
del teamTwitter ,playerTwitter
gc.collect();

In [ ]:
# Обогащаем командной целью
train = train.merge(team_tag_sum, on=['teamId', 'date'], how='left')
#train.fillna({'numberOfFollowers':0, 'twit_trend':0}, inplace=True)
train.fillna({'targ_team':0, 'targ_team_mean':0}, inplace=True)
train['targ_team'].value_counts()
del team_tag_sum
gc.collect();

In [ ]:
# Нормировка
# from sklearn.preprocessing import StandardScaler , MinMaxScaler
# scaler = MinMaxScaler()
# col_train = ['numberOfFollowers' ,'twit_med_f' ,'twit_min_f' ,'twit_max_f']
# scaler.fit(train[col_train])
# train[col_train] = scaler.transform(train[col_train])
# train.head()

In [ ]:
# Обогащаем годом рождения
train =  udate_year_old(train)
# train['date_year'] = pd.to_datetime(train['date'], format = '%Y%m%d')
# train['date_year'] = pd.DatetimeIndex(train['date_year']).year
# newColumn = lambda x: x['date_year'] - x['birthday_year']
# train['year_old'] = train.apply(newColumn, axis=1)

In [ ]:
train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])

In [ ]:
# feature_cols = ['battingOrder', 'gamesPlayedBatting', 'flyOuts',
#        'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
#        'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
#        'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
#        'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
#        'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
#        'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
#        'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
#        'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
#        'groundOutsPitching', 'runsPitching', 'doublesPitching',
#        'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
#        'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
#        'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
#        'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
#        'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
#        'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
#        'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
#        'inheritedRunnersScored', 'catchersInterferencePitching',
#        'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
#        'assists', 'putOuts', 'errors', 'chances',
feature_cols = ["hits", "strikeOuts","homeRuns","runsScored","stolenBases", "strikeOutsPitching",
       "inningsPitched", "strikes", "flyOuts",  "groundOuts", "errors",
       'inSeason' , 'seasonPart',        
       'target1_mean', 'target1_median', 'target1_std', 'target1_min', 'target1_max', 'target1_prob',
       'target2_mean', 'target2_median', 'target2_std', 'target2_min', 'target2_max', 'target2_prob',
       'target3_mean', 'target3_median', 'target3_std', 'target3_min', 'target3_max', 'target3_prob',
       'target4_mean', 'target4_median', 'target4_std', 'target4_min', 'target4_max', 'target4_prob',
       'label_playerId', 'label_teamId', 'label_status','year_old','numberOfFollowers', 'twit_med_f' , 'twit_min_f', 'twit_max_f','twit_trend',
       # This +++++++++++ 
       'team_twit_med_f' , 'team_twit_min_f', 'team_twit_max_f','team_twit_trend','team_numberOfFollowers']
       #'label_playerId', 'label_teamId', 'label_status','year_old','twit_trend']         
feature_cols_2 = feature_cols +['target1','targ_team' ,'targ_team_mean']

In [ ]:
# label encoding
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)

# Here+++++++++++++++++++++++
# awar2num = {c: i for i, c in enumerate(train['awar_org'].unique())}
# train['label_awar'] = train['awar_org'].map(awar2num)
# train['label_awar'].value_counts()
# tran2num = {c: i for i, c in enumerate(train['tran_type'].unique())}
# train['label_tran'] = train['tran_type'].map(tran2num)
# train['label_tran'].value_counts()
# Here+++++++++++++++++++++++

In [ ]:
#train.head(10)

In [ ]:
# Test_split
train_X = train[feature_cols]
train_y = train[['target1', 'target2', 'target3', 'target4']]

_index = (train['date'] < 20210401)
x_train = train_X.loc[_index].reset_index(drop=True) 
y_train = train_y.loc[_index].reset_index(drop=True)
x_valid = train_X.loc[~_index].reset_index(drop=True)
y_valid = train_y.loc[~_index].reset_index(drop=True)

print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

In [ ]:
# Test_split
train_X_2 = train[feature_cols_2]
train_y = train[['target1', 'target2', 'target3', 'target4']]

_index = (train['date'] < 20210401)
x_train_2 = train_X_2.loc[_index].reset_index(drop=True)
y_train = train_y.loc[_index].reset_index(drop=True)
x_valid_2 = train_X_2.loc[~_index].reset_index(drop=True)
y_valid = train_y.loc[~_index].reset_index(drop=True)

print(x_train_2.shape)
print(x_valid_2.shape)
print(y_train.shape)
print(y_valid.shape)

In [ ]:
del train , train_X_2 ,train_X
gc.collect();

### Modeling

#### Model LGBM

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV , MultiTaskLassoCV
import matplotlib.pyplot as plt                  # plots
import seaborn as sns                            # more plots
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
import lightgbm as lgbm
from sklearn.metrics import mean_absolute_error
import pickle

In [ ]:
def fit_lgbm(x_train, y_train, x_valid, y_valid, target , params: dict=None, verbose=100):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    if os.path.isfile(f'../input/mbl-lgbm-test2/model_lgb_{target}.pkl'):
        with open(f'../input/mbl-lgbm-test2/model_lgb_{target}.pkl', 'rb') as fin:
            print('open', target)
            model = pickle.load(fin)
    else:
        model = lgbm.LGBMRegressor(**params)
        model.fit(x_train, y_train,eval_set=[(x_valid, y_valid)],early_stopping_rounds=verbose,verbose=verbose)
        with open(f'model_lgb_{target}.pkl', 'wb') as handle:
            pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
#     model = lgbm.LGBMRegressor(**params)
#     model.fit(x_train, y_train, 
#         eval_set=[(x_valid, y_valid)],  
#         early_stopping_rounds=verbose, 
#         verbose=verbose)
    oof_pred = model.predict(x_valid)
    score = mean_absolute_error(oof_pred, y_valid)
    print('mae:', score)
    return oof_pred, model, score

In [ ]:
params1 = {
   'objective':'mae',
   'reg_alpha': 0.14947461820098767, 
   'reg_lambda': 0.10185644384043743, 
   'n_estimators': 3633, 
   'learning_rate': 0.08046301304430488, 
   'num_leaves': 674, 
   'feature_fraction': 0.9101240539122566, 
   'bagging_fraction': 0.9884451442950513, 
   'bagging_freq': 8,
   'min_child_samples': 51
}

params2 = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 80,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 22
}

params4 = {
  'objective':'mae',
  'reg_alpha': 0.016468100279441976, 
  'reg_lambda': 0.09128335764019105,
  'n_estimators': 9868, 
  'learning_rate': 0.10528150510326864, 
  'num_leaves': 157, 
  'feature_fraction': 0.5419185713426886, 
  'bagging_fraction': 0.2637405128936662, 
  'bagging_freq': 19, 
  'min_child_samples': 71
}

params3 = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 10000,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 100
}

params = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 100000,
 'learning_rate': 0.1,
 'random_state': 42,
}



In [ ]:
oof1, model1, score1 = fit_lgbm(
    x_train, y_train['target1'],
    x_valid, y_valid['target1'],
    1,
    params1
    #params
)

oof2, model2, score2 = fit_lgbm(
    x_train_2, y_train['target2'],
    x_valid_2, y_valid['target2'],
    2,
    params2
    #params
)

oof3, model3, score3 = fit_lgbm(
    x_train_2, y_train['target3'],
    x_valid_2, y_valid['target3'],
    3,
    params3
    #params
)

oof4, model4, score4 = fit_lgbm(
    x_train_2, y_train['target4'],
    x_valid_2, y_valid['target4'],
    4,
    params4
    #params
)

score = (score1+score2+score3+score4) / 4
print(f'score: {score}')

### SUBMIT

In [ ]:
import mlb

In [ ]:
rosters_cols_test = ['playerId', 'teamId', 'status']
scores_cols_test = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']

In [ ]:
null = np.nan
true = True
false = False

import mlb
import copy
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test: # make predictions here
#     #+++++++++++++++++++++++++++++++
  
#     sub = copy.deepcopy(sample_prediction_df.reset_index())
#     #++++++++++++++++++++++++++++++
    sample_prediction_df = sample_prediction_df.reset_index(drop=True)
        
    # вынимаем код игрока из индекса
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
     # вынимаем дату из индекса
    sample_prediction_df['date'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[0]))
    sample_prediction_df['date'] = pd.to_datetime(sample_prediction_df['date'], format = '%Y%m%d')
    #display(sample_prediction_df)
   
    # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
    #display(test_df)
    
    if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
        test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
    else:
        test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in scores.columns:
            if col == 'playerId': continue
            test_scores[col] = np.nan
    
    test_scores = test_scores.groupby('playerId').sum().reset_index()
    
    # получаем айдишники игроков и даты
    test = sample_prediction_df[['playerId','date']].copy()

    #Here+++++++++++++++++++++++
#     if test_df['playerTwitterFollowers'].iloc[0] == test_df['playerTwitterFollowers'].iloc[0]:
#         test_twit_pl = pd.DataFrame(eval(test_df['playerTwitterFollowers'].iloc[0]))
#         test_twit_pl['period'] = pd.to_datetime(test_twit_pl['date'], format = '%Y%m%d')
#         test_twit_pl['twit_year'] = test_twit_pl['period'].dt.year
#         test_twit_pl['twit_month'] = test_twit_pl['period'].dt.month
#         test_twit_pl = udate_playTwiter(test_twit_pl)
#         test_twit_pl[col_train] = scaler.transform(test_twit_pl[col_train])
#     else:
#         test_twit_pl = playerTwitter.copy()
#         #test_twit_pl = test_twit_pl.drop(['med_f'],axis =1)
# #        test_twit_pl = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
# #         for col in playerTwitter.columns:
# #             if col == 'playerId': continue
# #             test_twit_pl[col] = np.nan
    
    #display(test_twit_pl)
    #test_twit_pl = udate_playTwiter(test_twit_pl)
    #display(test_twit_pl)
    
    test['twit_year'] = test['date'].dt.year
    #test['twit_month'] = test['date'].dt.month
    #display(test.head())
    test = test.merge(playerTwitter_test[playerTwitter_cols], on=['playerId', 'twit_year'], how='left')
    test.fillna({'numberOfFollowers':0, 'twit_med_f':0 , 'twit_min_f':0 ,'twit_max_f':0 ,'twit_trend':0 }, inplace=True)
    
     #test.fillna({'numberOfFollowers':0, 'twit_trend':0}, inplace=True)
    #display(test.head())
               
    #обогащаем данными игроков
    test = test.merge(players[players_cols], on='playerId', how='left')
    # обогощаем статусом
    test = test.merge(test_rosters[rosters_cols_test], on='playerId', how='left')
    
    #Here+++++++++++++++++++++++
    test = test.merge(teamTwitter_test[teamTwitter_cols], on=['teamId', 'twit_year'], how='left')
    test.fillna({'team_numberOfFollowers':0, 'team_twit_med_f':0 , 'team_twit_min_f':0 ,'team_twit_max_f':0 ,'team_twit_trend':0 }, inplace=True)
    #Here+++++++++++++++++++++++   
    
    # обогащаем периодом
    test_season = test.copy() 
    #display(test_season)
    test_season['period'] = test_season['date']
    test_season = test_season.set_index('period').to_period('D')
    test_season = udate_season(test_season)
    
    season_col = ['playerId','inSeason','seasonPart']
    test = test.merge(test_season[season_col], on=['playerId'], how='left')
    #display(test)
    # Обогащаем годом рождения
    test = udate_year_old(test)
    #display(test)
    
    #Here +++++++++++++++++++++++++
    # Обогащаем наградами
    #test = udate_award(test)
    #++++++++++++++++++++++++++++++
        
    #
    # обогащаем стат данными
    test = test.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])

    # обогащаем данными игры
    #test = test.merge(test_scores[scores_cols], on='playerId', how='left')
    test = test.merge(test_scores[scores_cols_test], on='playerId', how='left')
    
    #test = test.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
    test['label_teamId'] = test['teamId'].map(teamid2num)
    test['label_status'] = test['status'].map(status2num)
    #display(test)
    test_X = test[feature_cols]
    
    # predict
    pred1 = model1.predict(test_X)
    
    test['target1'] = np.clip(pred1,0,100)
    
    # here++++++++++++++++++++++++
    test_team_tag =  test.groupby(['label_teamId','date'])['target1'].agg('sum').reset_index()
    test_team_tag.columns = ['label_teamId', 'date','targ_team']
    test = test.merge(test_team_tag, on=['label_teamId', 'date'], how='left')
    test.fillna({'targ_team':0}, inplace=True)
    # here++++++++++++++++++++++++    
    
    test_X = test[feature_cols_2]
    
    pred2 = model2.predict(test_X)
    pred3 = model3.predict(test_X)
    pred4 = model4.predict(test_X)
    
    # merge submission
    #sample_prediction_df = sample_prediction_df.drop(['date'],axis =1)
    
    sample_prediction_df['target1'] = np.clip(pred1, 0, 100)
    sample_prediction_df['target2'] = np.clip(pred2, 0, 100)
    sample_prediction_df['target3'] = np.clip(pred3, 0, 100)
    sample_prediction_df['target4'] = np.clip(pred4, 0, 100)
    sample_prediction_df = sample_prediction_df.fillna(0.)
            
    #display(sample_prediction_df)
    del sample_prediction_df['playerId']
    del sample_prediction_df['date']
    #display(sample_prediction_df)
    env.predict(sample_prediction_df)
       
    # Update Available information
    